In [1]:
import anndata

In [35]:
import numpy as np
import pandas as pd
from scipy.stats import poisson

n_sample, n_gene, n_feature1 = 3000, 20, 2
X1 = np.random.normal(size=(n_sample, n_feature1)) # covariates
ground_truth = np.random.normal(size=(n_feature1, n_gene)) # feature x gene
beta = np.exp(X1 @ ground_truth) # cell x gene

# generate samples
Y = poisson(beta).rvs()
obs = pd.DataFrame(X1, columns=[f"dim{j}" for j in range(n_feature1)]) # cell x feature
adata = anndata.AnnData(X=Y, obs=obs)
adata

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 3000 × 20
    obs: 'dim0', 'dim1'

In [36]:
from scdesigner.minimal.scd3_instances import PoissonCopula

sim = PoissonCopula("~ dim0 + dim1 - 1", "~1")
sim.fit(adata, max_epochs=40, batch_size=512, lr=0.5)

Estimating copula covariance: 100%|██████████| 6/6 [00:00<00:00, 29.61it/s]


In [37]:
sim.parameters["marginal"]["mean"]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dim0,-0.448721,0.098312,-1.639157,0.090841,-0.404860,-0.557156,-0.266663,-0.213220,-1.295450,-0.958633,-1.365996,0.806618,0.707173,-0.330939,-0.319329,-0.858334,-0.551261,-1.097177,-0.335295,-1.105506
dim1,0.847583,-0.862767,2.297651,1.515982,0.163157,0.185847,0.141279,-0.091342,-0.955681,-0.351062,-0.909588,-0.556154,-0.319348,0.749439,-0.830797,-0.316012,-0.370087,2.127105,-0.922861,1.093009


In [38]:
pd.DataFrame(ground_truth)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.476824,0.121148,-1.641032,0.116917,-0.383862,-0.514326,-0.315808,-0.117626,-1.276432,-0.933343,-1.359314,0.834934,0.669020,-0.324009,-0.291092,-0.913605,-0.572613,-1.097302,-0.352984,-1.116543
1,0.846857,-0.870783,2.296606,1.522805,0.131295,0.207344,0.090926,-0.129014,-0.932778,-0.375431,-0.879484,-0.579307,-0.274671,0.799458,-0.788985,-0.351851,-0.379052,2.131506,-0.979099,1.096499


In [13]:
from scipy.stats import poisson, bernoulli

n_sample, n_gene, n_feature1, n_feature2 = 3000, 20, 2, 1
X1 = np.random.normal(size=(n_sample, n_feature1)) # beta covariates
X2 = np.random.normal(size=(n_sample, n_feature2)) # zero-inflation covariates
gt_beta = np.random.normal(size=(n_feature1, n_gene))
gt_pi = np.random.normal(size=(n_feature2, n_gene))
beta = np.exp(X1 @ gt_beta)
pi = 1 / (1 + np.exp(-(X2 @ gt_pi)))

# generate samples
Y = poisson(beta).rvs() * bernoulli(1 - pi).rvs()
obs1 = pd.DataFrame(X1, columns=[f"beta_dim{j}" for j in range(n_feature1)])
obs2 = pd.DataFrame(X2, columns=[f"pi_dim{j}" for j in range(n_feature2)])
obs = pd.concat([obs1, obs2], axis=1)
adata = anndata.AnnData(X=Y, obs=obs)
adata

/Users/pyl/anaconda3/envs/sc2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 3000 × 20
    obs: 'beta_dim0', 'beta_dim1', 'pi_dim0'

In [ ]:
from scdesigner.minimal.scd3_instances import ZeroInflatedPoissonCopula

zip_simulator = ZeroInflatedPoissonCopula(
    "~ beta_dim0 + beta_dim1 - 1",
    "~ pi_dim0 - 1",
    "~1"
)
zip_simulator.fit(adata, max_epochs = 40, batch_size=512, lr=0.5)

Estimating copula covariance: 100%|██████████| 6/6 [00:00<00:00, 23.91it/s]


In [32]:
zip_simulator.parameters['marginal']['mean']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
beta_dim0,-1.469000,-0.128094,-0.200916,-0.629739,-0.131428,0.055508,-1.148924,0.360861,-0.209684,-0.624081,-0.233787,1.361227,0.881712,-0.476569,0.552997,0.775021,0.965669,-0.293303,-0.109885,0.157222
beta_dim1,-1.237726,1.585366,-0.570216,-0.653664,-0.015551,-0.459196,-0.818069,2.143202,-0.532097,1.381722,-1.901045,-0.132929,0.373169,0.036474,-0.106267,0.556844,-0.080994,-1.054843,0.512368,2.266911


In [33]:
pd.DataFrame(gt_beta)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.516699,-0.068237,-0.232735,-0.728438,-0.130634,0.077719,-1.230093,0.498011,-0.219502,-0.64864,-0.341453,1.322284,0.859954,-0.496529,0.581607,0.814349,0.930081,-0.353208,-0.027672,0.175944
1,-1.229966,1.448228,-0.543479,-0.715892,-0.129413,-0.535788,-0.816410,2.216223,-0.476718,1.48647,-1.952968,-0.110114,0.356430,0.001881,-0.136349,0.601567,-0.125910,-1.129340,0.437240,2.186710
